# The inference of official baseline (Tensorflow + new tfrecords).

# Training code is [here](https://www.kaggle.com/wuliaokaola/getting-started-tpus-new-tfrecords).

In [ ]:
import os
import glob
import math
import numpy as np 
import pandas as pd 
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

In [ ]:
sub = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv', index_col=0)

In [ ]:
fs = sub.index

In [ ]:
def read_tfrecord(filename):
    filename = tf.strings.join(["../input/cassava-leaf-disease-classification/test_images/", filename])
    b = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(b)
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

dataset = tf.data.Dataset.from_tensor_slices(fs)
dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
dataset = dataset.prefetch(AUTO)
dataset = dataset.batch(1)


In [ ]:
model = tf.keras.models.load_model('../input/getting-started-tpus-new-tfrecords/resnet50.h5')
# model.summary(line_length=130)

In [ ]:
for i, image in enumerate(dataset.as_numpy_iterator()):
    ps = []
    ps.append(model.predict(image)[0])    
    ps = np.mean(ps, axis=0)
    sub.iloc[i, 0] = np.argmax(ps)

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
sub